# Neat Model Training Notebook
This notebook demonstrates a clean, modular workflow for image classification using Keras, including data loading, preprocessing, augmentation, model definition, training with callbacks, and final evaluation.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Directories
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'
IMG_HEIGHT, IMG_WIDTH = 48, 48
BATCH_SIZE = 64
EPOCHS = 50


ModuleNotFoundError: No module named 'pandas'

In [ ]:
def load_image_paths_and_labels(directory):
    image_paths, labels = [], []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for fname in os.listdir(label_dir):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(label_dir, fname))
                    labels.append(label)
    return pd.DataFrame({'path': image_paths, 'label': labels})

# Load dataframes
df_train = load_image_paths_and_labels(TRAIN_DIR)
df_test = load_image_paths_and_labels(TEST_DIR)


In [ ]:
# Split training into train and validation
df_train, df_val = train_test_split(df_train, test_size=0.2, stratify=df_train['label'], random_state=42)

# Data generators with augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from dataframe
train_gen = train_datagen.flow_from_dataframe(
    df_train, x_col='path', y_col='label', target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical', batch_size=BATCH_SIZE)
val_gen = val_datagen.flow_from_dataframe(
    df_val, x_col='path', y_col='label', target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical', batch_size=BATCH_SIZE)
test_gen = test_datagen.flow_from_dataframe(
    df_test, x_col='path', y_col='label', target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical', batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.3),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.3),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
print('Evaluating on test set:')
test_loss, test_acc = model.evaluate(test_gen)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Classification report
import sklearn.metrics as metrics
y_true = test_gen.classes
y_pred = np.argmax(model.predict(test_gen), axis=1)
print(metrics.classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))
